In [ ]:
ENV["JULIA_NUM_THREADS"] = 1

In [13]:
using Random
using LinearAlgebra
using Distributions

function initialize_parameters()
    # Parameters
    mu = 0.001 * [2.5, 1.5, 2.0]
    Sigma = 0.0008 * [
        3.0 -1.0 -0.5;
        -1.0 1.5 0.5;
        -0.5 0.5 2.0
    ]
    
    # Initial prices
    S0 = [0.01, 0.01, 0.01]
    transaction_cost_rates = [(0.0080, 0.0040), (0.0064, 0.0064), (0.0040, 0.0080)]
    
    return mu, Sigma, S0, transaction_cost_rates
end

function sigmoid(x::Float64)
    return 1 / (1 + exp(-x))
end

function get_cum_returns(mu::Vector{Float64}, Sigma::Matrix{Float64}, n::Int=5)
    return sum(rand(MvNormal(mu, Sigma), n), dims=2)
end

function generate_s_all(stepx::Float64)
    s_all = Vector{Float64}[]
    for x in 0:100
        for y in 0:100
            if all([10 <= x <= 90, 10 <= y <= 90, 10 <= 100-x-y <= 90])
                push!(s_all, [round(x*stepx, digits=2), round(y*stepx, digits=2)])
            end
        end
    end
    return s_all
end

function generate_a_all(stepx::Float64)
    a_all = Vector{Float64}[]
    for x in -10:10
        for y in -10:10
            if -11 < x + y < 11
                push!(a_all, [round(x*stepx, digits=2), round(y*stepx, digits=2)])
            end
        end
    end
    return a_all
end

function step!(s::Vector{Float64}, a::Vector{Float64}, log_portfolio_value_before::Float64, 
                transaction_cost_rates::Vector{Tuple{Float64, Float64}}, tc_rates::Vector{Float64})
    s = vcat(s, 1 - sum(s))
    a = vcat(a, -sum(a))
    portfolio_before = exp(log_portfolio_value_before) .* s
    r_t = get_cum_returns(mu, Sigma, 5)
    portfolio_inter = portfolio_before .* (1 .+ r_t)
    portfolio_value_inter = sum(portfolio_inter)
    weights_inter = portfolio_inter ./ portfolio_value_inter
    weights_after = s .+ a
    
    tc = zeros(3)
    for i in 1:3
        delta_weight = weights_after[i] - weights_inter[i]
        tc[i] = transaction_cost_rates[i][2 - (delta_weight > 0)] * abs(delta_weight) * portfolio_value_inter
    end
    
    log_portfolio_value_after = log(portfolio_value_inter - sum(tc))
    s_prime = round.(weights_after[1:2], digits=2)
    reward = sigmoid((log_portfolio_value_after - log_portfolio_value_before)*reward_const)
    return reward, s_prime, log_portfolio_value_after
end

function rsq_algorithm!(mu::Vector{Float64}, Sigma::Matrix{Float64}, S0::Vector{Float64}, 
                        transaction_cost_rates::Vector{Tuple{Float64, Float64}}, tc_rates::Vector{Float64},
                        s_all::Vector{Vector{Float64}}, a_all::Vector{Vector{Float64}},
                        K::Int, delta::Float64, H::Int, gamma::Float64, c_gamma::Float64, reward_const::Float64,
                        alpha_t::Float64, initial_portfolio_value::Float64, states::Int, actions::Int,
                        Q::Array{Float64, 3}, V::Matrix{Float64}, N::Matrix{Int}, res::Vector{Float64})
    b_precomputed = c_gamma * abs(exp(gamma*H) - 1) * sqrt(H * log((states*actions*K*H)/delta))
    
    st = time()
    for k in 1:K
        log_portfolio_value_before = 0.0
        s = s_all[rand(1:length(s_all))]
        for h in H:-1:1
            s_index = findfirst(isequal(s), s_all)
            allowed_actions_index = [i for i in 1:length(a_all) if 0.1 <= round(s[1] + a_all[i][1], digits=2) <= 0.9 && 0.1 <= round(s[2] + a_all[i][2], digits=2) <= 0.9 && 0.1 <= round(1 - s[1] - s[2] - a_all[i][1] - a_all[i][2], digits=2) <= 0.9]
            max_Q = maximum(Q[h+1, s_index, allowed_actions_index])
            a_index = allowed_actions_index[rand(findall(isequal(max_Q), Q[h+1, s_index, allowed_actions_index]))]
            N[s_index, a_index] += 1
            t = N[s_index, a_index]
            r, s_prime, log_portfolio_value_after = step!(s, a_all[a_index], log_portfolio_value_before, transaction_cost_rates, tc_rates)
            b = b_precomputed / sqrt(t)
            w = (1 - alpha_t) * exp(gamma * Q[h, s_index, a_index]) + alpha_t * exp(gamma * (r + V[h+1, findfirst(isequal(s_prime), s_all)]))
            Q[h+1, s_index, a_index] = 1/gamma * log(max(exp(gamma*(H-h+1)), w - alpha_t*b))
            V[h+1, s_index] = maximum(Q[h, s_index, :])
            log_portfolio_value_before = log_portfolio_value_after
            alpha_t = (H+1)/(H+h)
            s = s_prime
        end
        res[k] = log_portfolio_value_before
        if k % 2500 == 0
            println("Value (logscale): $log_portfolio_value_before, strategy: $s, iter: $k")
            et = time()
            println(et - st)
            st = et
        end
    end
end

# Introduce all constants
K = 50000
delta = 0.95
H = 250
gamma = -5.0
c_gamma = 0.1#0.0001
reward_const = 15.0
alpha_t = (H+1)/(H+1)
initial_portfolio_value = 1.0
stepx = 0.01
s_all = generate_s_all(stepx)
a_all = generate_a_all(stepx)
states = length(s_all)
actions = length(a_all)
Q = zeros(H+1, states, actions)
V = zeros(H+1, states)
for h in 0:H-1
    for s in 1:states
        V[h+1, s] = H-h-1
        for a in 1:actions
            Q[h+1, s, a] = H-h-1
        end
    end
end
N = zeros(Int, states, actions)
mu, Sigma, S0, transaction_cost_rates = initialize_parameters()
tc_rates = [x for tuple in transaction_cost_rates for x in tuple]
res = zeros(K)
rsq_algorithm!(mu, Sigma, S0, transaction_cost_rates, tc_rates, s_all, a_all, K, delta, H, gamma, c_gamma, reward_const, alpha_t, initial_portfolio_value, states, actions, Q, V, N, res)


Value (logscale): 3.0819802029414, strategy: [0.74, 0.16], iter: 2500
126.58100008964539
Value (logscale): 1.2689117476879974, strategy: [0.22, 0.54], iter: 5000
130.86699986457825
Value (logscale): 1.7239568334340412, strategy: [0.31, 0.16], iter: 7500
152.4869999885559
Value (logscale): 2.37685116266955, strategy: [0.31, 0.16], iter: 10000
170.39600014686584
Value (logscale): 1.214647764614225, strategy: [0.43, 0.27], iter: 12500
183.1449999809265
Value (logscale): 1.092561198483216, strategy: [0.23, 0.22], iter: 15000
186.06299996376038
Value (logscale): 2.2858397537383213, strategy: [0.49, 0.28], iter: 17500
201.49900007247925
Value (logscale): 1.8424637524902698, strategy: [0.13, 0.31], iter: 20000
230.29399991035461
Value (logscale): 2.5373855148755844, strategy: [0.19, 0.17], iter: 22500
150.3659999370575
Value (logscale): 2.508343151286958, strategy: [0.18, 0.49], iter: 25000
171.83700013160706


0.0016849221598716209
0.00020652299143785288
